In [1]:
import fasttext
import fasttext.util
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score


In [2]:
model = fasttext.load_model("../../Downloads/ubertext.fiction_news_wikipedia.filter_rus+short.tokens.txt.algo-cbow.epochs-15.subwords-2..5.neg_sampling-15.bin")


In [3]:
tsn_csv = "tsn_articles.csv"
ukrinform_csv = "ukrinform.csv"
pravda_csv = "pravda_articles.csv"
strana_csv = "0strana_articles.csv"
_112_csv = "112.csv"
newsone_csv = "newsone.csv"

true_news = [tsn_csv, pravda_csv, ukrinform_csv]
prop_news = [strana_csv, _112_csv, newsone_csv]

dr = "../news/csv/"

In [4]:
tsn = pd.read_csv(dr + tsn_csv, delimiter="\t", index_col=0)
pravda = pd.read_csv(dr + pravda_csv, delimiter="\t", index_col=0)
strana = pd.read_csv(dr + strana_csv, delimiter="\t", index_col=0)
_112 = pd.read_csv(dr + _112_csv, delimiter="\t", index_col=0)
newsone = pd.read_csv(dr + newsone_csv, delimiter="\t", index_col=0)
ukrinform = pd.read_csv(dr + ukrinform_csv, delimiter="\t", index_col=0)

In [5]:
strana.head()

,Article
number,
0,Російські нафта і газ підтримують позитивний б...
1,Фінансування Мінкульту не буде переглянуто у р...
2,Соратниця Санду пішла з виборів через скандал ...
3,"Супутникова система, що прибула в Україну. Фот..."
4,"контейнеровоз, що сів на мілину, шість днів бл..."


In [6]:
true_vectors = []
false_vectors = []
scaled_true_vectors = []
scaled_false_vectors = []
scaler = MinMaxScaler()
# df = pd.DataFrame()

for i, news in enumerate(true_news):
    true_content = pd.read_csv(dr + news, delimiter="\t")
#     if i < len(true_news)-1:
#         df[i] = true_content[" Article"].apply(lambda x: model.get_sentence_vector(str(x)))
#     else:
#         df[i] = true_content["Article"].apply(lambda x: model.get_sentence_vector(str(x)))
        
#     break
#     df = pd.concat([df, t])
    for t in true_content.iterrows():
        if i < len(true_news)-1:
            if pd.isna(t[1][" Article"]):
                continue
            true_vector = model.get_sentence_vector(t[1][" Article"])
        else:
            if pd.isna(t[1]["Article"]):
                continue
            true_vector = model.get_sentence_vector(t[1]["Article"])
            
        scaled_true_vector = scaler.fit_transform(true_vector.reshape(-1, 1))
        scaled_true_vectors.append(scaled_true_vector.squeeze(1))
        true_vectors.append(true_vector)
        
        
for i, news in enumerate(prop_news):
    false_content = pd.read_csv(dr + news, delimiter="\t")
    for f in false_content.iterrows():
        false_vector = model.get_sentence_vector(f[1]["Article"])
        scaled_false_vectors.append(scaler.fit_transform(false_vector.reshape(-1, 1)).squeeze(1))
        false_vectors.append(false_vector)


In [7]:
len(true_vectors), len(false_vectors)

(128587, 16417)

In [8]:
df = pd.DataFrame(true_vectors + false_vectors)
df['y'] = [0]*len(true_vectors) + [1]*len(false_vectors)
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,y
0,0.045944,0.033212,-0.019494,0.017835,0.017329,0.044720,0.036844,-0.013484,0.021245,0.006724,...,0.025386,0.011930,0.012249,-0.077265,0.017796,0.028253,0.015929,0.006496,-0.002924,0
1,0.059813,0.036721,-0.018274,0.032621,0.015537,0.051469,0.053786,-0.021347,0.017381,-0.009109,...,0.022771,0.015956,0.019941,-0.073319,0.021163,0.038026,0.010041,-0.003658,-0.010412,0
2,0.057720,0.031666,-0.015856,0.018313,0.018238,0.038563,0.043197,-0.020581,0.024852,-0.007280,...,0.020274,0.016442,0.019216,-0.061398,0.024590,0.027343,0.011330,-0.011836,0.008885,0
3,0.055922,0.037115,-0.009455,0.030506,0.023757,0.047180,0.052907,-0.034172,0.021569,-0.008985,...,0.020767,0.021013,0.023593,-0.070772,0.012401,0.030589,-0.000591,-0.005169,-0.008243,0
4,0.055116,0.015518,-0.011605,0.043243,0.007095,0.042123,0.033189,-0.015345,0.020922,-0.022534,...,0.020315,0.002096,0.032026,-0.066096,0.002333,0.044644,0.008299,-0.004881,-0.010561,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144999,0.057816,0.030768,-0.017235,0.026246,0.021929,0.042852,0.046335,-0.022112,0.019113,-0.003631,...,0.030071,0.014059,0.017375,-0.067576,0.025667,0.042140,-0.001227,0.004661,-0.006640,1
145000,0.053990,0.032169,-0.017542,0.023458,0.020964,0.043306,0.043331,-0.019090,0.019162,-0.002829,...,0.030633,0.011633,0.017338,-0.068067,0.022882,0.040825,0.000284,0.005336,-0.006218,1
145001,0.055371,0.029537,-0.017212,0.021284,0.018991,0.044607,0.042242,-0.019062,0.018636,-0.003438,...,0.030407,0.011757,0.016129,-0.067525,0.020673,0.039090,0.001660,0.002860,-0.005084,1
145002,0.058424,0.026798,-0.017176,0.010653,0.023448,0.047812,0.047891,-0.018901,0.022463,-0.004640,...,0.026067,0.015758,0.020223,-0.073846,0.022762,0.031064,0.004096,-0.001435,-0.004549,1


In [9]:
scaled_df = pd.DataFrame(scaled_true_vectors + scaled_false_vectors)
scaled_df['y'] = [0]*len(scaled_true_vectors) + [1]*len(scaled_false_vectors)
scaled_df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,y
0,0.859938,0.771070,0.403213,0.663752,0.660218,0.851392,0.796426,0.445160,0.687548,0.586198,...,0.716448,0.622536,0.624759,0.000000,0.663480,0.736461,0.650444,0.584611,0.518864,0
1,0.844904,0.714645,0.404429,0.691514,0.595152,0.797832,0.810905,0.387098,0.605551,0.456125,...,0.635954,0.597511,0.619991,0.093935,0.626885,0.722005,0.564150,0.486877,0.448778,0
2,0.894345,0.720890,0.404503,0.631990,0.631487,0.766807,0.797656,0.373044,0.675522,0.461599,...,0.645046,0.619534,0.638002,0.101301,0.673779,0.692105,0.585495,0.431270,0.569223,0
3,0.857668,0.749213,0.480662,0.711104,0.672182,0.807257,0.840279,0.338130,0.659568,0.483372,...,0.654942,0.656361,0.671236,0.127073,0.606700,0.711580,0.531778,0.505378,0.487653,0
4,0.776037,0.557951,0.408572,0.710646,0.511566,0.704477,0.655277,0.387975,0.587713,0.348383,...,0.584374,0.484033,0.648868,0.108468,0.485335,0.718365,0.518196,0.445604,0.414325,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144999,0.893894,0.721787,0.416355,0.693019,0.665546,0.798681,0.820841,0.385326,0.647630,0.502913,...,0.717354,0.615475,0.636572,0.096046,0.689334,0.794145,0.518214,0.555678,0.483770,1
145000,0.879524,0.740036,0.422251,0.684346,0.668401,0.811225,0.811384,0.412353,0.656882,0.516305,...,0.730212,0.608754,0.645226,0.099261,0.680663,0.795365,0.536202,0.568503,0.494637,1
145001,0.890083,0.723015,0.420679,0.669639,0.654809,0.820472,0.805175,0.408712,0.652513,0.509758,...,0.728638,0.608027,0.636299,0.095291,0.665686,0.784796,0.542727,0.550485,0.499114,1
145002,0.906241,0.702938,0.420260,0.599155,0.681401,0.838022,0.838533,0.409170,0.675071,0.500842,...,0.698242,0.631967,0.660673,0.055967,0.676994,0.730363,0.557002,0.521449,0.501431,1


In [10]:
X, y = df.iloc[:, :-1], df['y']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [17]:
scaled_X, scaled_y = scaled_df.iloc[:, :-1], scaled_df['y']

scaled_X_train, scaled_X_test, scaled_y_train, scaled_y_test = train_test_split(
    scaled_X, scaled_y, test_size=0.2, random_state=42)

In [24]:
def count_metrics(y_test, prediction):
    y = y_test.to_numpy()

    conf = confusion_matrix(prediction, y)
    acc = accuracy_score(prediction, y)
    precision = precision_score(prediction, y, zero_division=1)
    recall = recall_score(prediction, y, zero_division=1)
    f1 = f1_score(prediction, y, zero_division=1)
    balanced_acc = balanced_accuracy_score(prediction, y, adjusted=False)

    print("Confusion matrix:\n", conf)
    print("Accuracy: ", acc)
    print("Balanced accuracy: ", balanced_acc)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", f1)

### Unscaled vectors

In [26]:
clf = GaussianNB().fit(X_train, y_train)
prediction = clf.predict(X_test)
count_metrics(y_test, prediction)

Confusion matrix:
 [[17849   891]
 [ 7851  2410]]
Accuracy:  0.6985621185476363
Balanced accuracy:  0.5936622690988259
Precision:  0.7300817933959406
Recall:  0.23486989572166456
F1:  0.3554048075505088


In [27]:
forest = RandomForestClassifier(max_depth=5, random_state=42).fit(X_train, y_train)
prediction_forest = forest.predict(X_test)
count_metrics(y_test, prediction_forest)

Confusion matrix:
 [[25700  3300]
 [    0     1]]
Accuracy:  0.8862108203165409
Balanced accuracy:  0.943103448275862
Precision:  0.0003029385034837928
Recall:  1.0
F1:  0.0006056935190793458


### Scaled vectors

In [28]:
clf = GaussianNB().fit(scaled_X_train, scaled_y_train)
prediction = clf.predict(scaled_X_test)
count_metrics(scaled_y_test, prediction)

Confusion matrix:
 [[13885   933]
 [11815  2368]]
Accuracy:  0.5604289507258371
Balanced accuracy:  0.5519982414279414
Precision:  0.7173583762496213
Recall:  0.16696044560389198
F1:  0.2708762296957218


In [29]:
clf = MultinomialNB().fit(scaled_X_train, scaled_y_train)
prediction = clf.predict(scaled_X_test)
count_metrics(scaled_y_test, prediction)

Confusion matrix:
 [[25700  3301]
 [    0     0]]
Accuracy:  0.8861763387469398
Balanced accuracy:  0.8861763387469398
Precision:  0.0
Recall:  1.0
F1:  0.0


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1953: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [30]:
forest = RandomForestClassifier(max_depth=5, random_state=42).fit(scaled_X_train, scaled_y_train)
prediction_forest = forest.predict(scaled_X_test)
count_metrics(scaled_y_test, prediction_forest)

Confusion matrix:
 [[25700  3282]
 [    0    19]]
Accuracy:  0.8868314885693597
Balanced accuracy:  0.9433786488165068
Precision:  0.005755831566192063
Recall:  1.0
F1:  0.01144578313253012
